In [5]:
import requests
import time
import pandas as pd
import os

TOKEN = "d5c3d78fc111d88a0a37b4ab8f83cbd5"
url = "https://ows.goszakup.gov.kz/v3/subject/all"
parquet_file = "goszakup_subjAll_data.parquet"

headers = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}

params = {"limit": 500}  
all_data = []

# Проверяем, есть ли уже сохраненные данные
if os.path.exists(parquet_file):
    df_existing = pd.read_parquet(parquet_file, engine="pyarrow")
    last_update_date = df_existing["last_update_date"].max()  
    print(f"Последняя дата обновления: {last_update_date}")
else:
    last_update_date = None

while True:
    try:
        if last_update_date:
            params.update({"last_update_date": last_update_date})

        response = requests.get(url, headers=headers, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        items = data.get("items", [])
        if not items:
            print("Новых данных нет, загрузка завершена.")
            break

        new_items = [item for item in items if item["last_update_date"] > last_update_date]
        if not new_items:
            print("Нет новых записей.")
            break

        all_data.extend(new_items)
        print(f"Загружено {len(all_data)} новых записей...")

        last_update_date = max(item["last_update_date"] for item in new_items)

        # Промежуточное сохранение
        if len(all_data) % 1000 < 500:
            df_new = pd.DataFrame(all_data)
            df_combined = pd.concat([df_existing, df_new], ignore_index=True)
            df_combined.to_parquet(parquet_file, index=False, engine="pyarrow")
            print("Промежуточное сохранение...")

        time.sleep(5)

    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса: {e}")
        print("Ждём 30 сек и пробуем снова...")
        time.sleep(30)

# Финальное сохранение
if all_data:
    df_new = pd.DataFrame(all_data)
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    df_combined.to_parquet(parquet_file, index=False, engine="pyarrow")
    print(f"Дозагрузка завершена, новые данные добавлены в {parquet_file}")
else:
    print("Новых данных не найдено.")

Последняя дата обновления: 2025-03-16 15:54:06
Ошибка запроса: HTTPSConnectionPool(host='ows.goszakup.gov.kz', port=443): Read timed out.
Ждём 30 сек и пробуем снова...
Загружено 500 новых записей...
Ошибка запроса: HTTPSConnectionPool(host='ows.goszakup.gov.kz', port=443): Read timed out.
Ждём 30 сек и пробуем снова...
Нет новых записей.
Дозагрузка завершена, новые данные добавлены в goszakup_subjAll_data.parquet


In [6]:
df = pd.read_parquet("goszakup_subjAll_data.parquet")
print(df.shape)
df.head()

(584304, 45)


,pid,bin,iin,inn,unp,regdate,crdate,number_reg,series,name_ru,...,mark_world_company,mark_state_monopoly,mark_natural_monopoly,mark_patronymic_producer,mark_patronymic_supplyer,mark_small_employer,type_supplier,is_single_org,system_id,index_date
0,771220,,020529600732,,,None,2025,,,ИП Сулушаш,...,0,0,0,0,0,0,3,0,3,2025-03-16 12:20:02
1,771219,,810111350069,,,None,2025,,,ИП МУХАТОВ УСЕНАЛИ ШАКУАЛИЕВИЧ,...,0,0,0,0,0,0,2,0,3,2025-03-16 11:40:02
2,771218,,890327450718,,,None,2025,,,ИП Жаксыбаева,...,0,0,0,0,0,0,2,0,3,2025-03-16 11:20:01
3,771217,,710929301851,,,None,2025,,,МЫНЖАСАРОВ АСХАТ,...,0,0,0,0,0,0,2,0,3,2025-03-16 11:00:02
4,771216,,800826400301,,,None,2025,,,ОСПАНОВА,...,0,0,0,0,0,0,3,0,3,2025-03-16 10:40:02
